In [1]:
%matplotlib notebook
%load_ext autoreload
%autoreload 1
!hostname
!pwd

dv001.ib.bridges2.psc.edu
/ocean/projects/asc170022p/mtragoza/mre-pinn/notebooks


In [2]:
import sys, os
import numpy as np

sys.path.append('..')
%aimport mre_pinn

sys.path.append('../../param_search')
%aimport param_search
ps = param_search

Using backend: pytorch



# Single-frequency 2D

In [7]:
mre_pinn.data.load_bioqic_dataset('../data/BIOQIC', 'fem_box')[0].frequency

Loading ../data/BIOQIC/four_target_phantom.mat
    __header__: <class 'bytes'>
    __version__: <class 'str'>
    __globals__: <class 'list'>
    u_ft: <class 'numpy.ndarray'> (100, 80, 10, 3, 6) complex128
Loading ../data/BIOQIC/fem_box_elastogram.npy
     <class 'numpy.ndarray'> (6, 10, 80, 100) complex128
Loading ../data/BIOQIC/fem_box_regions.npy
     <class 'numpy.ndarray'> (10, 80, 100) int64
Multi frequency 3D
<xarray.Dataset>
Dimensions:         (frequency: 6, x: 80, y: 100, z: 10, component: 3)
Coordinates:
  * frequency       (frequency) float64 50.0 60.0 70.0 80.0 90.0 100.0
  * x               (x) float64 0.0 0.001 0.002 0.003 ... 0.077 0.078 0.079
  * y               (y) float64 0.0 0.001 0.002 0.003 ... 0.097 0.098 0.099
  * z               (z) float64 0.0 0.001 0.002 0.003 ... 0.007 0.008 0.009
  * component       (component) <U1 'z' 'y' 'x'
Data variables:
    u               (frequency, x, y, z, component) complex128 (7.43590828448...
    mu              (frequency, x,

<xarray.DataArray 'frequency' (frequency: 6)>
array([ 50.,  60.,  70.,  80.,  90., 100.])
Coordinates:
  * frequency  (frequency) float64 50.0 60.0 70.0 80.0 90.0 100.0

In [10]:
# define the job template and name format

template = '''\
#!/bin/bash
#SBATCH --job-name={job_name}
#SBATCH --account=bio170034p
#SBATCH --partition=BatComputer
#SBATCH --gres=gpu:rtx5000:1
#SBATCH --time=48:00:00
#SBATCH -o %J.stdout
#SBATCH -e %J.stderr
#SBATCH --mail-type=all

hostname
pwd
source activate MRE-PINN

python ../../../train.py \\
    --data_root ../../../data/BIOQIC \\
    --data_name fem_box \\
    --frequency {frequency} \\
    --xyz_slice {xyz_slice} \\
    --pde_name {pde_name} \\
    --omega0 {omega0} \\
    --n_layers {n_layers} \\
    --n_hidden {n_hidden} \\
    --activ_fn {activ_fn} \\
    --learning_rate {learning_rate} \\
    --pde_loss_wt {pde_loss_wt} \\
    --data_loss_wt {data_loss_wt} \\
    --batch_size {batch_size} \\
    --n_domain {n_domain} \\
    --n_iters {n_iters} \\
    --test_every {test_every} \\
    --save_every {save_every} \\
    --save_prefix {job_name}    
'''
name = 'train_{frequency}_{xyz_slice}_{pde_name}_{omega0}_{n_hidden}_{activ_fn}'

# define the parameter space

param_space = ps.ParamSpace(
    frequency=[50, 60, 70, 80, 90, 100],
    xyz_slice=['2D'],
    pde_name=['hetero'],
    omega0=[8, 16, 32],
    n_layers=5,
    n_hidden=16,
    activ_fn=['s', 't'],
    learning_rate=1e-4,
    pde_loss_wt=1,
    data_loss_wt=1e-8,
    batch_size=80,
    n_domain=128-80,
    pde_distrib='uniform',
    n_iters=100000,
    test_every=1000,
    save_every=10000
)

len(param_space)

36

In [11]:
%autoreload
expt_name = '2022-07-24_2D'

jobs = ps.submit(template, name, param_space, work_dir=expt_name, verbose=True)
jobs.to_csv(f'{expt_name}.jobs')

#jobs = pd.read_csv(f'{expt_name}.jobs')

100%|██████████| 36/36 [00:00<00:00, 137.46it/s]
[9951778, 9951779, 9951780, 9951781, 9951782, 9951783, 9951784, 9951785, 9951786, 9951787, 9951788, 9951789, 9951790, 9951791, 9951792, 9951793, 9951794, 9951795, 9951796, 9951797, 9951798, 9951799, 9951800, 9951801, 9951802, 9951803, 9951804, 9951805, 9951806, 9951807, 9951808, 9951809, 9951810, 9951811, 9951812, 9951813]


In [13]:
status_cols = ['job_name', 'job_state', 'node_id', 'runtime', 'stdout', 'stderr']
ps.status(jobs)[status_cols]

,job_name,job_state,node_id,runtime,stdout,stderr
job_id,,,,,,
9951778,train_50_2D_hetero_8_16_s,RUNNING,dv002,1:59,NaN,NaN
9951779,train_50_2D_hetero_8_16_t,RUNNING,dv002,1:59,NaN,NaN
9951780,train_50_2D_hetero_16_16_s,RUNNING,dv003,1:59,NaN,NaN
9951781,train_50_2D_hetero_16_16_t,RUNNING,dv003,1:59,NaN,NaN
9951782,train_50_2D_hetero_32_16_s,RUNNING,dv001,1:59,NaN,NaN
9951783,train_50_2D_hetero_32_16_t,RUNNING,dv001,1:59,NaN,NaN
9951784,train_60_2D_hetero_8_16_s,RUNNING,dv001,1:59,NaN,NaN
9951785,train_60_2D_hetero_8_16_t,RUNNING,dv001,1:59,NaN,NaN
9951786,train_60_2D_hetero_16_16_s,RUNNING,dv002,1:16,NaN,NaN
